In [384]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import re
from io import StringIO

# 1.0- Data Scraping

First we must pull the data from baskeball-reference.com

In [470]:
lebron = 'https://www.basketball-reference.com/players/j/jamesle01.html'
rose = 'https://www.basketball-reference.com/players/r/rosede01.html'
mj = 'https://www.basketball-reference.com/players/j/jordami01.html'
z = 'https://www.basketball-reference.com/players/z/zipsepa01.html'
players = [lebron,rose,mj,z]

In [471]:
chromedriver = "/home/michael/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [523]:
driver = webdriver.Chrome(chromedriver)
driver.get(players[2])
time.sleep(3)
expand = driver.find_element_by_xpath('//button[@id="meta_more_button"]')
time.sleep(1)
expand.click()

## 1.1 Get the Player Info

In [524]:
name = driver.find_element_by_xpath('//h1[@itemprop="name"]').text
name

'Michael Jordan'

In [539]:
height = driver.find_element_by_xpath('//span[@itemprop="height"]').text
height = height.split('-')
height = int(height[0])+(int(height[1])/12)
height

6.5

In [540]:
weight = driver.find_element_by_xpath('//span[@itemprop="weight"]').text
weight = int(weight[:-2])
weight

195

In [527]:
info = driver.find_element_by_xpath('//div[@id="info"]').text
info = info.split('\n')
info #10 Recruit 11 Draft 

['Michael Jordan',
 'Michael Jeffrey Jordan',
 '(Mike, Air Jordan, M.J., His Airness, Money, Black Cat, G.O.A.T., Superman)',
 'Position: Small Forward and Shooting Guard ▪ Shoots: Right',
 '6-6, 195lb (198cm, 88kg)',
 'Born: February 17, 1963 (Age: 54-342d) in Brooklyn, New York us',
 'College: University of North Carolina',
 'High School: Emsley A. Laney in Wilmington, North Carolina',
 'Draft: Chicago Bulls, 1st round (3rd pick, 3rd overall), 1984 NBA Draft',
 'NBA Debut: October 26, 1984',
 'Hall of Fame: Inducted as Player in 2009 (Full List)',
 'Hall of Fame',
 '14x All Star',
 '10x NBA Scoring Champ',
 '6x NBA Champ',
 '11x All-NBA',
 '9x All-Defensive',
 '1984-85 All-Rookie',
 '3x AS MVP',
 '1987-88 Def. POY',
 '6x Finals MVP',
 '5x MVP',
 '1984-85 ROY',
 '23',
 '12',
 '45',
 '23',
 'Support us without the ads? Go Ad-Free.',
 'SUMMARY',
 'Career',
 'G',
 '1072',
 'PTS',
 '30.1',
 'TRB',
 '6.2',
 'AST',
 '5.3',
 'FG%',
 '49.7',
 'FG3%',
 '32.7',
 'FT%',
 '83.5',
 'eFG%',
 '50.9'

In [528]:
draft_regex = re.compile('Draft:')
draftstr = [x  for x in info if draft_regex.search(x)]
draftstr = draftstr[0].split(',')
draftstr

['Draft: Chicago Bulls',
 ' 1st round (3rd pick',
 ' 3rd overall)',
 ' 1984 NBA Draft']

In [529]:
draftnbr = draftstr[2].strip().split(' ')
draftnbr = draftnbr[0]
draftnbr = int(draftnbr[:-2])
draftnbr

3

In [530]:
draftyr = draftstr[3].strip().split(' ')
draftyr = int(draftyr[0])
draftyr

1984

In [548]:
try:
    Trecruit_regex = re.compile('Recruiting')
    recruitstr = [x  for x in info if recruit_regex.search(x)]    
    recruitstr = recruitstr[0].split(' ')
    recruitnbr = recruitstr[-1]
    recruitnbr = int(recruitnbr[1:-1])
except:
    recruitnbr = 0
    
recruitnbr

0

In [555]:
dfscrapeinfo = pd.DataFrame(index=[0])
dfscrapeinfo['player'] = name
dfscrapeinfo['height'] = height
dfscrapeinfo['weight'] = weight
dfscrapeinfo['draftnbr']  = draftnbr
dfscrapeinfo['draftyr'] = draftyr
dfscrapeinfo['recruitnbr'] = recruitnbr
dfscrapeinfo

,player,height,weight,draftnbr,draftyr,recruitnbr
0,Michael Jordan,6.5,195,3,1984,0


## 1.2 Get the VORP Yearly Log

In [532]:
advstatsxpath = driver.find_element_by_xpath('//table[@id="advanced"]').text
advstatsxpath = advstatsxpath.replace('  ','')
advstatsxpath

'Season Age Tm Lg Pos G MP PER TS% 3PAr FTr ORB% DRB% TRB% AST% STL% BLK% TOV% USG% OWS DWS WS WS/48 OBPM DBPM BPM VORP\n1984-85 21 CHI NBA SG 82 3144 25.8 .592 .032 .459 6.3 13.2 9.8 25.3 3.0 1.3 13.0 29.8 10.3 3.7 14.0 .213 6.8 1.4 8.2 8.1\n1985-86 22 CHI NBA SG 18 451 27.5 .533 .055 .381 5.6 10.7 8.0 21.7 3.9 2.7 10.5 38.6 1.0 0.5 1.5 .160 5.1 -0.5 4.7 0.8\n1986-87 23 CHI NBA SG 82 3281 29.8 .562 .029 .427 5.6 9.3 7.4 22.2 3.6 2.3 9.1 38.3 11.9 5.0 16.9 .247 8.0 0.6 8.6 8.8\n1987-88 24 CHI NBA SG 82 3311 31.7 .603 .027 .430 4.8 10.7 7.8 27.0 3.9 2.4 9.6 34.1 15.2 6.1 21.2 .308 9.8 2.3 12.2 11.8\n1988-89 25 CHI NBA SG 81 3255 31.1 .614 .055 .442 5.5 17.3 11.6 34.7 3.6 1.2 11.9 32.1 14.6 5.2 19.8 .292 9.8 2.7 12.6 12.0\n1989-90 26 CHI NBA SG 82 3197 31.2 .606 .125 .356 5.3 15.6 10.4 28.6 3.5 1.1 9.8 33.7 14.7 4.3 19.0 .285 9.7 0.8 10.6 10.1\n1990-91 27 CHI NBA SG 82 3034 31.6 .605 .051 .365 4.6 14.3 9.5 25.2 3.7 1.7 8.7 32.9 14.9 5.4 20.3 .321 8.9 1.8 10.8 9.8\n1991-92 28 CHI NBA SG 8

In [533]:
dfadvstats = pd.read_csv(StringIO(advstatsxpath),sep=' ')
dfadvstats = dfadvstats.dropna()
dfadvstats['player'] = name
dfadvstats

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,player
0,1984-85,21,CHI,NBA,SG,82.0,3144.0,25.8,0.592,0.032,...,29.8,10.3,3.7,14.0,0.213,6.8,1.4,8.2,8.1,Michael Jordan
1,1985-86,22,CHI,NBA,SG,18.0,451.0,27.5,0.533,0.055,...,38.6,1.0,0.5,1.5,0.160,5.1,-0.5,4.7,0.8,Michael Jordan
2,1986-87,23,CHI,NBA,SG,82.0,3281.0,29.8,0.562,0.029,...,38.3,11.9,5.0,16.9,0.247,8.0,0.6,8.6,8.8,Michael Jordan
3,1987-88,24,CHI,NBA,SG,82.0,3311.0,31.7,0.603,0.027,...,34.1,15.2,6.1,21.2,0.308,9.8,2.3,12.2,11.8,Michael Jordan
4,1988-89,25,CHI,NBA,SG,81.0,3255.0,31.1,0.614,0.055,...,32.1,14.6,5.2,19.8,0.292,9.8,2.7,12.6,12.0,Michael Jordan
5,1989-90,26,CHI,NBA,SG,82.0,3197.0,31.2,0.606,0.125,...,33.7,14.7,4.3,19.0,0.285,9.7,0.8,10.6,10.1,Michael Jordan
6,1990-91,27,CHI,NBA,SG,82.0,3034.0,31.6,0.605,0.051,...,32.9,14.9,5.4,20.3,0.321,8.9,1.8,10.8,9.8,Michael Jordan
7,1991-92,28,CHI,NBA,SG,80.0,3102.0,27.7,0.579,0.055,...,31.7,12.1,5.6,17.7,0.274,6.9,1.7,8.6,8.3,Michael Jordan
8,1992-93,29,CHI,NBA,SG,78.0,3067.0,29.7,0.564,0.115,...,34.7,12.0,5.2,17.2,0.270,8.3,1.2,9.5,8.9,Michael Jordan
9,1994-95,31,CHI,NBA,SG,17.0,668.0,22.1,0.493,0.079,...,33.2,1.2,1.1,2.3,0.167,2.0,0.7,2.7,0.8,Michael Jordan


## 1.3 College Stats

In [534]:
collegexpath = driver.find_element_by_xpath('//table[@id="all_college_stats"]').text
#collegexpath = collegexpath.replace('  ','')
collegexpath

'Totals Per Game\nSeason Age College G MP FG FGA 3P 3PA FT FTA ORB TRB AST STL BLK TOV PF PTS FG% 3P% FT% MP PTS TRB AST\n1981-82 18 UNC 34 191 358 78 108 149 61 41 8 57 91 460 .534 .722 13.5 4.4 1.8\n1982-83 19 UNC 36 282 527 34 76 123 167 197 56 78 28 76 110 721 .535 .447 .737 20.0 5.5 1.6\n1983-84 20 UNC 31 247 448 113 145 163 64 50 35 67 70 607 .551 .779 19.6 5.3 2.1\nCareer 101 720 1333 34 76 314 420 509 181 169 71 200 271 1788 .540 .447 .748 17.7 5.0 1.8'

In [543]:
df = pd.DataFrame()
df

""


In [547]:
df2 = df.append(dfadvstats)
df2

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,player
0,1984-85,21,CHI,NBA,SG,82.0,3144.0,25.8,0.592,0.032,...,29.8,10.3,3.7,14.0,0.213,6.8,1.4,8.2,8.1,Michael Jordan
1,1985-86,22,CHI,NBA,SG,18.0,451.0,27.5,0.533,0.055,...,38.6,1.0,0.5,1.5,0.160,5.1,-0.5,4.7,0.8,Michael Jordan
2,1986-87,23,CHI,NBA,SG,82.0,3281.0,29.8,0.562,0.029,...,38.3,11.9,5.0,16.9,0.247,8.0,0.6,8.6,8.8,Michael Jordan
3,1987-88,24,CHI,NBA,SG,82.0,3311.0,31.7,0.603,0.027,...,34.1,15.2,6.1,21.2,0.308,9.8,2.3,12.2,11.8,Michael Jordan
4,1988-89,25,CHI,NBA,SG,81.0,3255.0,31.1,0.614,0.055,...,32.1,14.6,5.2,19.8,0.292,9.8,2.7,12.6,12.0,Michael Jordan
5,1989-90,26,CHI,NBA,SG,82.0,3197.0,31.2,0.606,0.125,...,33.7,14.7,4.3,19.0,0.285,9.7,0.8,10.6,10.1,Michael Jordan
6,1990-91,27,CHI,NBA,SG,82.0,3034.0,31.6,0.605,0.051,...,32.9,14.9,5.4,20.3,0.321,8.9,1.8,10.8,9.8,Michael Jordan
7,1991-92,28,CHI,NBA,SG,80.0,3102.0,27.7,0.579,0.055,...,31.7,12.1,5.6,17.7,0.274,6.9,1.7,8.6,8.3,Michael Jordan
8,1992-93,29,CHI,NBA,SG,78.0,3067.0,29.7,0.564,0.115,...,34.7,12.0,5.2,17.2,0.270,8.3,1.2,9.5,8.9,Michael Jordan
9,1994-95,31,CHI,NBA,SG,17.0,668.0,22.1,0.493,0.079,...,33.2,1.2,1.1,2.3,0.167,2.0,0.7,2.7,0.8,Michael Jordan
